In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [124]:
df=pd.read_csv("/home/user456/Downloads/data.csv")
df

,Name,Gender,Count,Probability
0,James,M,5304407,1.451679e-02
1,John,M,5260831,1.439753e-02
2,Robert,M,4970386,1.360266e-02
3,Michael,M,4579950,1.253414e-02
4,William,M,4226608,1.156713e-02
...,...,...,...,...
147264,Zylenn,M,1,2.736740e-09
147265,Zymeon,M,1,2.736740e-09
147266,Zyndel,M,1,2.736740e-09
147267,Zyshan,M,1,2.736740e-09


In [125]:
print(len(df["Name"].unique()))

133910


In [126]:
df=df.drop_duplicates(subset=["Name"])
df=df.drop(columns=["Count","Probability"])
df

,Name,Gender
0,James,M
1,John,M
2,Robert,M
3,Michael,M
4,William,M
...,...,...
147264,Zylenn,M
147265,Zymeon,M
147266,Zyndel,M
147267,Zyshan,M


In [127]:
class model():
    def __init__(self):
        self.male={}
        self.female={}
        self.total_male=0
        self.total_female=0
    def predict(self,name):
        self.name=name.lower().replace(" ","")
        #building of p(name|gender)
        pm=0
        pf=0
        worldsize = len(set(self.male.keys()).union(self.female.keys(), set(self.name))) #for random chars as i just saw lol


        for x in self.name:
            pm += math.log((self.male.get(x, 0) + 1) / (self.total_male + worldsize))   #hopefully correct laplace
            pf += math.log((self.female.get(x, 0) + 1) / (self.total_female + worldsize))

        #p(gender|name) proportional to p(name|gender)*p(gender)
        #as per latest 0.505 and 0.495 but i will take that
        fpm = pm + math.log(0.505)
        fpf = pf + math.log(0.495)

        if fpm > fpf:
            return "M", math.exp(fpm) / (math.exp(fpm) + math.exp(fpf))
        else:
            return "F", math.exp(fpf) / (math.exp(fpm) + math.exp(fpf))

    def train(self,name,gender):
        if gender=="M":
            for x in name:
                if x in self.male:
                    self.male[x]+=1
                else:
                    self.male[x]=1
            self.total_male+=len(name)
        else:
            for x in name:
                if x in self.female:
                    self.female[x]+=1
                else:
                    self.female[x]=1
            self.total_female+=len(name)
    def __str__(self):
        return f"Male:{self.male}\nFemale:{self.female}"
    
    

In [128]:
m1=model()

In [129]:
#training the model
for row in df.values:
    
    m1.train(row[0].lower().replace(" ",""),row[1])
print(m1)

Male:{'j': 6995, 'a': 45004, 'm': 11234, 'e': 31670, 's': 16094, 'o': 19804, 'h': 14129, 'n': 28592, 'r': 25070, 'b': 5366, 't': 12896, 'i': 24874, 'c': 7111, 'l': 16780, 'w': 2669, 'd': 13011, 'v': 5027, 'p': 2908, 'y': 9621, 'g': 4348, 'u': 9635, 'k': 8585, 'f': 2228, 'x': 924, 'z': 3228, 'q': 1124, "'": 91, '…': 7, '.': 55, '-': 1826, 'œ': 1, '@': 54, ',': 4, '0': 3, ';': 3, '?': 3, '1': 1, '9': 2, '5': 2, '7': 2, 'ö': 1, '/': 1, '(': 1, ')': 1, '[': 1, '8': 1}
Female:{'m': 17379, 'a': 108753, 'r': 34427, 'y': 24280, 'e': 67559, 'l': 40694, 'i': 53535, 'z': 4751, 'b': 6340, 't': 21504, 'h': 26202, 'p': 3251, 'c': 11255, 'j': 8242, 'n': 51743, 'f': 2410, 'd': 13948, 'g': 5183, 's': 26489, 'u': 9316, 'o': 16295, 'k': 12668, 'v': 5427, 'q': 1344, 'x': 882, 'w': 2222, '-': 5431, 'à': 3, '.': 56, '…': 6, '0': 2, 'œ': 1, "'": 163, '@': 47, ',': 5, '/': 5, '"': 2, ';': 5, '¡': 1, '&': 1, '(': 1}


In [130]:
print(m1.predict("John Wick"))
print(m1.predict("Alexander"))

('M', 0.872261209889407)
('F', 0.5007031602893381)


Ahh this works for some but fails maybe i better add p(name at char i|gender)
Not sure how it will work but let's do this

In [131]:
class model2():
    def __init__(self):
        self.male={}
        self.female={}
        self.total_male=0
        self.total_female=0
    def predict(self,name):
        self.name=name.lower().replace(" ","")
        #building of p(name|gender)
        pm=0
        pf=0
        worldsize = len(set(self.male.keys()).union(self.female.keys(), set(self.name))) #ehh now adds positional too
        for i,x in enumerate(self.name):
            x=str(i)+x
            pm += math.log((self.male.get(x, 0) + 1) / (self.total_male + worldsize))   #hopefully correct laplace
            pf += math.log((self.female.get(x, 0) + 1) / (self.total_female + worldsize))

        #p(gender|name) proportional to p(name|gender)*p(gender)
        #as per latest 0.505 and 0.495 but i will take that
        fpm = pm + math.log(0.505)
        fpf = pf + math.log(0.495)

        if fpm > fpf:
            return "M", math.exp(fpm) / (math.exp(fpm) + math.exp(fpf))
        else:
            return "F", math.exp(fpf) / (math.exp(fpm) + math.exp(fpf))

    def train(self,name,gender):
        if gender=="M":
            for i,x in enumerate(name):
                x=str(i)+x
                if x in self.male:
                    self.male[x]+=1
                else:
                    self.male[x]=1
            self.total_male+=len(name)
        else:
            for i,x in enumerate(name):
                x=str(i)+x
                if x in self.female:
                    self.female[x]+=1
                else:
                    self.female[x]=1
            self.total_female+=len(name)
    def __str__(self):
        return f"Male:{self.male}\nFemale:{self.female}"
    
    

In [132]:
m2=model2()

In [133]:
#training the model
for row in df.values:
    
    m2.train(row[0].lower().replace(" ",""),row[1])
print(m2)

Male:{'0j': 4698, '1a': 15064, '2m': 2700, '3e': 4754, '4s': 1977, '1o': 5037, '2h': 1892, '3n': 3763, '0r': 2718, '2b': 998, '4r': 3344, '5t': 1510, '0m': 3340, '1i': 4401, '2c': 1436, '3h': 2478, '4a': 6101, '5e': 3820, '6l': 1562, '0w': 721, '2l': 2882, '3l': 2307, '4i': 4757, '5a': 3559, '6m': 451, '0d': 3931, '2v': 1595, '3i': 4874, '4d': 1406, '2s': 2715, '4p': 257, '5h': 1219, '5r': 2942, '6d': 958, '0c': 2545, '1h': 3126, '2a': 5334, '3r': 3261, '4l': 2359, '6s': 1078, '0t': 3235, '2o': 1711, '3m': 1671, '5s': 1540, '2r': 6200, '6o': 1707, '7p': 109, '8h': 348, '9e': 397, '10r': 155, '2n': 4083, '4e': 5999, '5l': 2437, '2t': 1725, '3t': 2282, '4h': 1741, '6w': 156, '0a': 4463, '1n': 917, '4o': 3322, '5n': 6570, '6y': 597, '0g': 1550, '1e': 6944, '4g': 404, '3a': 6058, '5d': 1196, '0p': 1184, '2u': 1172, '3k': 1296, '2d': 1914, '5w': 187, '0s': 3555, '1t': 582, '2e': 3472, '3v': 1225, '0k': 3674, '6h': 877, '0e': 1697, '1d': 573, '2w': 532, '4u': 1428, '0b': 2334, '1r': 3645, '2

In [134]:
print(m2.predict("John Wick"))
print(m2.predict("Alexander"))

('M', 0.9533016302699129)
('M', 0.5988811402746541)


Wooooooo it works gg!!!!!!!!
But the values are very less and smoothing although small may do some issue but i think this is good enough